<a href="https://colab.research.google.com/github/PRihana/Machine-learning/blob/master/09_04_2018(ML)DTC%26_Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("/content/car.data.txt",names = \
                   ['buying',"maint","doors","persons","lug","safety","class"])
data.head()

,buying,maint,doors,persons,lug,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [0]:
data.dtypes

buying     object
maint      object
doors      object
persons    object
lug        object
safety     object
class      object
dtype: object

In [0]:
x = data.drop("class",axis=1)
y = data['class']

In [0]:
y.value_counts()

unacc    1210
acc       384
good       69
vgood      65
Name: class, dtype: int64

In [0]:
y[(y=='good') | (y=='vgood')] = "acc"
y.value_counts()

unacc    1210
acc       518
Name: class, dtype: int64

In [0]:
y.apply(lambda x:1 if x=="unacc" else 0)

0       1
1       1
2       1
3       1
4       1
       ..
1723    0
1724    0
1725    1
1726    0
1727    0
Name: class, Length: 1728, dtype: int64

In [9]:
# Don't run this cell multiple times
y = y.apply(lambda x:1 if x=="unacc" else 0)
y

0       1
1       1
2       1
3       1
4       1
       ..
1723    0
1724    0
1725    1
1726    0
1727    0
Name: class, Length: 1728, dtype: int64

In [10]:
y.value_counts()

1    1210
0     518
Name: class, dtype: int64

In [11]:
x

,buying,maint,doors,persons,lug,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med
...,...,...,...,...,...,...
1723,low,low,5more,more,med,med
1724,low,low,5more,more,med,high
1725,low,low,5more,more,big,low
1726,low,low,5more,more,big,med


In [12]:
x1 = pd.get_dummies(x)
x1

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,doors_4,doors_5more,persons_2,persons_4,persons_more,lug_big,lug_med,lug_small,safety_high,safety_low,safety_med
0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1
2,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0
3,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0
4,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1
1724,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0
1725,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0
1726,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1


In [13]:
x.nunique().sum()

21

In [14]:
x2 = x.copy()
from sklearn.preprocessing import LabelEncoder
LR = LabelEncoder()
for col in x.columns:
    x2[col]=LR.fit_transform(x[col])
x2

,buying,maint,doors,persons,lug,safety
0,3,3,0,0,2,1
1,3,3,0,0,2,2
2,3,3,0,0,2,0
3,3,3,0,0,1,1
4,3,3,0,0,1,2
...,...,...,...,...,...,...
1723,1,1,3,2,1,2
1724,1,1,3,2,1,0
1725,1,1,3,2,0,1
1726,1,1,3,2,0,2


In [15]:
y1 = data['class']
y1[(y1=='good') | (y1=='vgood')] = "acc"
y1

0       unacc
1       unacc
2       unacc
3       unacc
4       unacc
        ...  
1723      acc
1724      acc
1725    unacc
1726      acc
1727      acc
Name: class, Length: 1728, dtype: object

In [16]:
# x,y1
from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB()
mb.fit(x2,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
mb.predict(x2.iloc[1:2,])

array([1])

In [18]:
mb.predict(x2)

array([1, 1, 1, ..., 0, 1, 0])

In [19]:
y1

0       unacc
1       unacc
2       unacc
3       unacc
4       unacc
        ...  
1723      acc
1724      acc
1725    unacc
1726      acc
1727      acc
Name: class, Length: 1728, dtype: object

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y,mb.predict(x2))

0.7204861111111112

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y,mb.predict(x2))

array([[  73,  445],
       [  38, 1172]])

In [22]:
from sklearn.metrics import precision_score
precision_score(y,mb.predict(x2))

0.7247990105132962

In [23]:
from sklearn.metrics import f1_score
f1_score(y,mb.predict(x2))

0.8291475061903077

In [24]:
from sklearn.naive_bayes import BernoulliNB
mb = BernoulliNB()
mb.fit(x1,y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [25]:
f1_score(y,mb.predict(x1))

0.9647558386411891

In [26]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x2,y)
y_pred = dtc.predict(x2)
f1_score(y,y_pred)

1.0

In [27]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion="entropy",max_depth=5)
dtc.fit(x2,y)
y_pred = dtc.predict(x2)
f1_score(y,y_pred)

0.9556285949055052

In [28]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=7,max_features=4)
dtc.fit(x2,y)
y_pred = dtc.predict(x2)
f1_score(y,y_pred)

0.9717656974294142

In [29]:
help(DecisionTreeClassifier)

Help on class DecisionTreeClassifier in module sklearn.tree._classes:

class DecisionTreeClassifier(sklearn.base.ClassifierMixin, BaseDecisionTree)
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : {"gini", "entropy"}, default="gini"
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  splitter : {"best", "random"}, default="best"
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to choose the best split and "random" to choose
 |      the best random split.
 |  
 |  max_depth : int, default=None
 |      The maximum depth of the tree. If None, then nodes are expanded until
 |      all leaves are pure or until all leaves contain less than
 |      min_samples_split samples.
 |  
 |  min_samples_split : int or float, default=2
 |      The minimum 

In [30]:
from sklearn.ensemble import BaggingClassifier
BC = BaggingClassifier(n_estimators=5)
BC.fit(x2,y)
y_pred = BC.predict(x2)
f1_score(y,y_pred)

0.9995866060355519